In [ ]:
!pip install pandas folium geopy pycountry

In [34]:
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
import time

# === 1. Загрузка данных ===
df = pd.read_csv('covid_19_clean_complete.csv')
df['Date'] = pd.to_datetime(df['Date'])

# === 2. Группировка по дате и стране ===
grouped = df.groupby(['Date', 'Country/Region']).agg({
    'Confirmed': 'sum', 'Lat': 'mean', 'Long': 'mean'
}).reset_index()

# === 3. Объединяем координаты ===
# Координаты берём прямо из исходного файла (это ускоряет и избавляет от геокодирования!)
grouped = grouped.rename(columns={'Lat':'lat', 'Long':'lon'})
grouped = grouped.dropna(subset=['lat', 'lon'])

# === 4. Глобальный максимум для нормализации ===
global_max = grouped['Confirmed'].max()
dates_sorted = grouped['Date'].sort_values().unique()

# === 5. Формируем данные для HeatMapWithTime ===
heat_data = []
for date in dates_sorted:
    day_data = grouped[grouped['Date'] == date]
    heat_day = [
        [row['lat'], row['lon'], row['Confirmed'] / global_max]
        for idx, row in day_data.iterrows() if row['Confirmed'] > 0
    ]
    heat_data.append(heat_day)

# === 6. Создаём карту ===
m = folium.Map(location=[20,0], zoom_start=2, tiles='CartoDB dark_matter')

HeatMapWithTime(
    heat_data,
    index=[str(d.date()) for d in dates_sorted],
    auto_play=True,
    max_opacity=0.8,
    gradient={
        0.2: 'blue',
        0.4: 'lime',
        0.6: 'yellow',
        0.8: 'orange',
        1.0: 'red'
    }
).add_to(m)

m.save('covid_global_heatmap.html')
print("Готово! Откройте файл covid_global_heatmap.html в браузере.")


Готово! Откройте файл covid_global_heatmap.html в браузере.


In [35]:
m

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from geopy.geocoders import Nominatim
import time

# === 1. Загрузка и подготовка данных ===
df = pd.read_csv('covid_19_clean_complete.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Группировка по стране и дате
grouped = df.groupby(['Date', 'Country/Region'])['Confirmed'].sum().reset_index()

# --- Геокодирование стран (по необходимости — смотри кэширование чтобы ускорить) ---
geolocator = Nominatim(user_agent="covid_map")
coords_cache = {}

def get_coords(country):
    if country in coords_cache:
        return coords_cache[country]
    try:
        loc = geolocator.geocode(country)
        if loc:
            coords_cache[country] = (loc.latitude, loc.longitude)
            return coords_cache[country]
    except:
        pass
    coords_cache[country] = (np.nan, np.nan)
    return (np.nan, np.nan)

# Получаем координаты (если очень долго — советую один раз получить и сохранить кэш)
grouped['lat'] = grouped['Country/Region'].apply(lambda x: get_coords(x)[0])
grouped['lon'] = grouped['Country/Region'].apply(lambda x: get_coords(x)[1])

# Удаляем страны без координат
grouped = grouped.dropna(subset=['lat', 'lon'])

# --- Добавляем признаки времени ---
grouped['dayofweek'] = grouped['Date'].dt.dayofweek
grouped['month'] = grouped['Date'].dt.month

# --- Целевая переменная: новые случаи на след. день ---
grouped = grouped.sort_values(['Country/Region', 'Date'])
grouped['Confirmed_next'] = grouped.groupby('Country/Region')['Confirmed'].shift(-1)
grouped['target_new_cases'] = grouped['Confirmed_next'] - grouped['Confirmed']
grouped = grouped.dropna(subset=['target_new_cases'])
grouped = grouped[grouped['target_new_cases'] >= 0]

# --- Формируем признаки и цель ---
features = ['lat', 'lon', 'Confirmed', 'dayofweek', 'month']
X = grouped[features]
y = grouped['target_new_cases']

# === 2. train_test_split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

# === 3. Обучение модели и проверка ===
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE на тесте: {rmse:.2f}")

# === 4. Сохраняем модель для FastAPI/Streamlit ===
import joblib
joblib.dump(model, "model.pkl")
print("Модель сохранена в model.pkl")


Train size: (26007, 5), Test size: (6502, 5)
RMSE на тесте: 570.87
Модель сохранена в model.pkl


In [ ]:
import pandas as pd
import numpy as np
import joblib
import folium

# Загружаем модель
model = joblib.load('model.pkl')
df = pd.read_csv('covid_19_clean_complete.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Определяем целевую дату для прогноза
target_date = pd.to_datetime('2020-08-01')

# Находим последнюю известную дату и сколько шагов до цели
last_known_date = df['Date'].max()
horizon = (target_date - last_known_date).days

if horizon <= 0:
    print("Указанная дата есть в датасете! Можно визуализировать реальные данные или обычный прогноз.")
else:
    # Готовим стартовые значения
    grouped = df[df['Date'] == last_known_date].groupby('Country/Region').agg({
        'Confirmed': 'sum', 'Lat': 'mean', 'Long': 'mean'
    }).reset_index()
    grouped['lat'] = grouped['Lat']
    grouped['lon'] = grouped['Long']

    # Итеративно прогнозируем каждый следующий день до нужной даты
    for i in range(1, horizon + 1):
        pred_date = last_known_date + pd.Timedelta(days=i)
        grouped['dayofweek'] = pred_date.dayofweek
        grouped['month'] = pred_date.month
        features = ['lat', 'lon', 'Confirmed', 'dayofweek', 'month']
        X_pred = grouped[features]
        grouped['predicted_new_cases'] = model.predict(X_pred)
        # Обновляем Confirmed для следующего дня
        grouped['Confirmed'] = grouped['Confirmed'] + grouped['predicted_new_cases']

    m = folium.Map(location=[20, 0], zoom_start=2, tiles='CartoDB dark_matter')
    for _, row in grouped.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=max(2, np.log1p(row['predicted_new_cases'])),
            color='red' if row['predicted_new_cases'] > 0 else 'blue',
            fill=True,
            fill_opacity=0.6,
            popup=f"{row['Country/Region']}: {int(row['predicted_new_cases'])} новых случаев"
        ).add_to(m)


Карта прогноза на 2020-08-01 сохранена!


In [39]:
m